In [2]:
import pandas as pd
import numpy as np


data = pd.read_csv('data/data-logistic.csv', header=None, names=['target', 'f1', 'f2'])
X = data[['f1', 'f2']]
y = data['target']
del data

In [26]:
w01 = 0
w02 = 0
i = 0
print(1 - 1 / (1 + np.exp(-y[i]*(w01*X.iloc[i]['f1'] + w02*X.iloc[i]['f2']))))

0.5


In [30]:
w01 = 0
w02 = 0
w11 = 0
w12 = 0
k = 0.1
err = 1e-5
l = X.shape[0]
i = 1
for _ in range(0, 10000):
    grad_sum1 = 0
    grad_sum2 = 0
    for j in range(0, l):
        grad_sum1 += y[j] * X.iloc[j][0] * (1 - 1 / (1 + np.exp(-y[j] * (w01 * X.iloc[j][0] + w02 * X.iloc[j][1]))))
        grad_sum2 += y[j] * X.iloc[j][1] * (1 - 1 / (1 + np.exp(-y[j] * (w01 * X.iloc[j][0] + w02 * X.iloc[j][1]))))
    w11 += (1 / l) * k * grad_sum1
    w12 += (1 / l) * k * grad_sum2    
    if np.linalg.norm(np.array([w01 - w11, w02 - w12])) <= err:
        print('good')
        break
    w01 = w11
    w02 = w12

    # w1 = w0 + (1 / l) * k * np.sum(np.outer(y, np.dot(X.transpose(), 1 + (1 / (1 + np.exp(-y * np.dot(X, w0)))))), 
    #                                axis=0)    
    # if np.linalg.norm(w1 - w0) <= err:
    #     break
    # if i == 10000:
    #     print(0)
    #     break
    # w0 = np.copy(w1)
    # i += 1
print(w01)
print(w02)

good
0.287804423488
0.0919899571942


In [31]:
c = 10
rw0 = np.array([0, 0])
i = 1
while True:
    rw1 = rw0 + (1 / l) * k * np.sum(np.outer(y, np.dot(X.transpose(), (1 + (1 / (1 + np.exp(-y * np.dot(X, rw0))))))), 
                                   axis=0) - k * c * rw0    
    if np.linalg.norm(rw1 - rw0) <= err:
        break
    if i == 10000:
        print(1)
        break
    rw0 = np.copy(rw1)
    i += 1
print(rw1)
print(rw0)

[ 0.7853851   0.76377707]
[ 0.78538402  0.76377612]


In [7]:
from sklearn.metrics import roc_auc_score

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

ryt = sigmoid(np.dot(X, rw1))
yt = sigmoid(np.dot(X, w1))
ans = np.array([np.round(roc_auc_score(y, yt), 3), 
                np.round(roc_auc_score(y, ryt), 3)])
print(ans)
np.savetxt('output/week3/task3.txt', ans.transpose(), newline=' ', fmt='%-.3f')

[ 0.716  0.936]


/home/morgolt/Projects/Coursera/iml/mlvenv/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
